# LSTM
Long Short-Term Memory 长的短期记忆

## 输入输出
### 在t时刻, LSTM的输入
1. $X_t$ 当前时刻网络的输入
2. $H_{t-1}$ 前一时刻LSTM的输出
3. $C_{t-1}$ 前一时刻的单元状态(独创之处)(短期的记忆可以一直传递下去)

### 在t时刻, LSTM的输出
1. $h_t$ 当前时刻LSTM的输入
2. $C_t$ 当前时刻的单元状态

## 词向量编码
- 类似"聚类", 有相关性的单词距离近

## 一些术语的概念
- Batch size : 批次(样本)数目。一次迭代（Forword 运算（用于得到损失函数）以及 BackPropagation 运算（用于更新神经网络参数））所用的样本数目。Batch size 越大，所需的内存就越大
- Iteration : 迭代。每一次迭代更新一次权重（网络参数），每一次权重更新需要 Batch size 个数据进行 Forward 运算，再进行 BP 运算
- Epoch : 纪元/时代。所有的训练样本完成一次迭代
    - 对于Epoch大小的确定，牵扯到了防止过拟合的一个方法：提前停止训练。随着epoch次数增加，神经网络中的权重的更新次数也增加，模型从欠拟合变得过拟合。
    - trick：可以先设定一个固定的Epoch大小（100轮）一般当模型的loss不再持续减小，且精度不在10轮内提升，就可以提前停止训练了。（设置条件来停止epoch）

假如 : 训练集有 1000 个样本，Batch_size=10
那么 : 训练完整个样本集需要： 100 次 Iteration，1 个 Epoch
但一般我们都不止训练一个 Epoch

## 超参数（Hyper parameter）
- init_scale : 权重参数（Weights）的初始取值跨度，一开始取小一些比较利于训练
- learning_rate : 学习率，训练时初始为 1.0
- num_layers : LSTM 层的数目（默认是 2）
- num_steps : LSTM 展开的步（step）数，相当于每个批次输入单词的数目（默认是 35）
- hidden_size : LSTM 层的神经元数目，也是词向量的维度（默认是 650）
- max_lr_epoch : 用初始学习率训练的 Epoch 数目（默认是 10）, 过了这个训练次数后, 学习率开始衰减
- dropout : 在 Dropout 层的留存率（默认是 0.5）. 没有留下来的不参与运算, 防止过拟合 
- lr_decay : 在过了 max_lr_epoch 之后每一个 Epoch 的学习率的衰减率，训练时初始为 0.93。让学习率逐渐衰减是提高训练效率的有效方法
- batch_size : 批次(样本)数目。一次迭代（Forword 运算（用于得到损失函数）以及 BackPropagation 运算（用于更新神经网络参数））所用的样本数目（batch_size 默认是 20。取比较小的 batch_size 更有利于 Stochastic Gradient Descent（随机梯度下降），防止被困在局部最小值）
    - 当有足够算力时，选取batch size为32或更小一些。
    - 算力不够时，在效率和泛化性之间做trade-off，尽量选择更小的batch size。
    - 当模型训练到尾声，想更精细化地提高成绩（比如论文实验/比赛到最后），有一个有用的trick，就是设置batch size为1，即做纯SGD，慢慢把error磨低
    - [【调参炼丹】 Batch_size和Epoch_size](https://blog.csdn.net/qq_38358305/article/details/88643163#commentsedit)


In [15]:
# PTB数据集
import os
import datetime
import sys
import argparse
import collections

import numpy as np
import tensorflow as tf

data_path = "../resource/data/simple-examples/data"
with tf.gfile.GFile("../resource/data/simple-examples/data/ptb.test.txt", "r") as f:
    f = f.read().replace("\n", "<eos>").split()
    counter = collections.Counter(f)
    print(counter)

Counter({'<unk>': 4794, 'the': 4529, '<eos>': 3761, 'N': 2523, 'of': 2195, 'to': 2042, 'a': 1821, 'in': 1640, 'and': 1539, "'s": 903, 'that': 831, 'for': 783, 'is': 667, 'said': 601, '$': 564, 'it': 542, 'on': 507, 'as': 461, 'by': 450, 'at': 420, 'with': 405, 'its': 393, 'was': 391, 'be': 384, 'are': 365, 'from': 355, 'million': 348, "n't": 335, 'but': 335, 'mr.': 319, 'have': 309, 'he': 302, 'market': 300, 'has': 296, 'will': 294, 'an': 260, 'about': 258, 'this': 255, 'company': 227, 'or': 220, 'they': 220, 'new': 219, 'year': 214, 'were': 203, 'which': 202, 'more': 198, 'would': 189, 'u.s.': 179, 'stock': 175, 'had': 175, 'than': 173, 'friday': 171, 'their': 171, 'been': 163, 'says': 158, 'one': 148, 'not': 147, 'up': 146, 'corp.': 144, 'some': 140, 'we': 137, 'billion': 134, 'his': 133, 'also': 133, 'prices': 132, 'when': 131, 'i': 130, 'share': 129, 'last': 128, 'inc.': 128, 'other': 127, 'who': 120, '&': 118, 'all': 113, 'sales': 113, 'because': 112, 'if': 110, 'after': 109, 'pre

In [31]:
# PTB数据集
import os
import datetime
import sys
import argparse
import collections

import numpy as np
import tensorflow as tf

data_path = "../resource/data/simple-examples/data"
parser = argparse.ArgumentParser() # 参数解析器
parser.add_argument('--data_path', type=str, default=data_path, help='The path of the data for training and testing') # 用来指定程序需要接受的命令参数
args = parser.parse_args()

# 词向量编码, 按照词频给编码

# 分割
def read_words(filename):
    with tf.io.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split() # 分成一个个单词, 从空格断, 从句末断

# 构造从单词到唯一整数值的映射
# 后面的其他数的整数值按照它们在数据集里出现的次数多少来排序，出现较多的排前面
# 单词 the 出现频次最多，对应整数值是 0
# <unk> 表示 unknown（未知），第二多，整数值为 1  
def build_vocab(filename):
    data = read_words(filename)
    
    counter = collections.Counter(data)
    counter_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0])) # 先按数值从大到小排, 再按单词首字母排

    # zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的对象，这样做的好处是节约了不少的内存. 可以使用list()转换成列表. 利用 * 号操作符，可以将元组解压为列表
    words, _ = list(zip(*counter_pairs)) # 只要排序后的单词

    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id

# 将文件里的单词都替换成独一的整数
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

# 加载所有数据，读取所有单词，把其转成唯一对应的整数值
def load_data():
    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    word_to_id = build_vocab(train_path)

    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id) #?? 没有对应数值的单词怎么办
    test_data = file_to_word_ids(test_path, word_to_id)

    # 所有不重复单词的个数
    vocab_size = len(word_to_id)

    # 反转一个词汇表：为了之后从 整数 转为 单词
    id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(word_to_id)
    print("===================")
    print(vocab_size)
    print("===================")
    print(train_data[:10])
    print("===================")
    print(" ".join([id_to_word[x] for x in train_data[:10]]))
    print("===================")
    return train_data, valid_data, test_data, vocab_size, id_to_word

if __name__ == "__main__":
    load_data()

usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\sucon\AppData\Roaming\jupyter\runtime\kernel-c1eef573-0575-48df-94b1-9a54a99eb4e9.json


SystemExit: 2